# Building a deep learning model with tensorflow keras for non linear classification problem
 
There are four steps to build and use a machine learning model.

- **Preprocessing** : feature selection, ramdomize, normalize, shuffle, EDA, split train test
- **Learning** : implementing the model
- **Evaluation** : metrics ( true positive, false positive...etc), tests, overfitting undertitting, hyper tunnin
- **Prediction** on new unseen data

## Problem context

You will implement a deep learning model to predict whether microchips from a fabrication plant passes quality assurance (QA). During QA, each microchip goes through various tests to ensure it is functioning correctly.
Suppose you are the product manager of the factory and you have the test results for some microchips on two different tests. From these two tests, you would like to determine whether the microchips should be accepted or rejected. To help you make the decision, you have a dataset of test results on past microchips, from which you can build a logistic regression model.

## 1 -  Preprocessing et EDA

La première étape consiste à charger puis visualizer puis analyser les données

In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#fonction utilitaire pour afficher les points

def plot_points(X, y):
    sns.scatterplot(X[:, 0], X[:, 1], hue=y.flatten(), cmap=plt.cm.coolwarm, s=30, edgecolors='k')
    plt.show()
    
# fonction utilitaire pour randomiser les données (shuffle)
def randomize(X, Y):
    permutation = np.random.permutation(Y.shape[0])
    X2 = X[permutation,:]
    Y2 = Y[permutation]
    Y2[Y2 == -1] = 0 
    return X2, Y2  

# fonction utilitaire pour split les données en test et train sets
def split_train_test(data, test_size):
    sample = np.random.choice(data.index, size=int(len(data)*(1 - test_size)), replace=False)
    train_data, test_data = data.iloc[sample], data.drop(sample)
    return train_data, test_data 

### Load data

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/hzitoun/workshop-deep-learning-tensorflow/master/microships_QA.csv');
data.head()

### Split data into train and test sets

In order to test our algorithm, we'll split the data into a Training and a Testing set.

In [ ]:
train_data, test_data = split_train_test(data, test_size=0.1)

### Extract features and target columns

In [ ]:
X_train = np.array(train_data.drop('result', axis=1))
y_train = np.array(train_data['result']).reshape(-1, 1)
X_test = np.array(test_data.drop('result', axis=1))
y_test = np.array(test_data['result']).reshape(-1, 1)

print("Shape of X_train = ", X_train.shape, "Shape of y_train=", y_train.shape, "\n")
print("Shape of X_test = ",  X_test.shape, "Shape of y_test =",  y_test.shape, "\n")

### Randomize features and target columns

In [ ]:
X_train, y_train = randomize(X_train, y_train)
X_test, y_test = randomize(X_test, y_test)

### Visualize dataset

In [ ]:
plot_points(X_train, y_train)

## 2 -  Implementing the deep learning model in tf.keras

In [ ]:
# TODO import tensorflow and keras layers


In [ ]:
print(tf.VERSION)
print(tf.keras.__version__)

**Get Reproducible Results with Keras by Seeding Random Numbers with the TensorFlow Backend**

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
# Create the Sequential model
model =  tf.keras.Sequential()

# TODO

In [ ]:
# TODO compile the model using mean squared error as loss and stochastic gradient descent as optimizer, and auccracy for metrics


In [ ]:
model.summary()

In [ ]:
tb = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, 
                                 write_graph=True, 
                                 write_grads=False, write_images=False, 
                                 embeddings_freq=0, 
                                 embeddings_layer_names=None, 
                                 embeddings_metadata=None, 
                                 embeddings_data=None, update_freq='epoch')

# TODO fit the model using X_train and y_train, with a batch size of 1 and tensboard as a callback

## 3 -  Evaluate 

Once we've trained our neural network, we need to evaluate our model on test data: accuracy, false positive, true positive

In [ ]:
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy: {:.3f}%".format(accuracy * 100))

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

### Tensorboard

tensorboard --logdir ./logs/

open http://localhost:6006

In [ ]:
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_descision_boundary(estimator, X, y):
    X0, X1 = X[:, 0], X[:, 1]
    xx, yy = make_meshgrid(X0, X1)
    target = np.c_[xx.ravel(), yy.ravel()]
    Z = (estimator.predict(target) > 0.5).astype(int)
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
    sns.scatterplot(X0, X1, hue=y.flatten(), cmap=plt.cm.coolwarm, s=30, edgecolors='k')
    plt.show()

**Plot the descision boundary to get a visualization of what our neural networks have learned.**

In [ ]:
plot_descision_boundary(model, X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
y_pred = (model.predict(X_test) > 0.5).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('confusion matrix', confusion_matrix(y_test, y_pred))
print('tn,fp,fn,tp \n', tn, fp, fn, tp)
print('precision', precision_score(y_test, y_pred))
print('recall', recall_score(y_test, y_pred))
print('f1 score', f1_score(y_test,y_pred))

## 4 -  Predict on new unseen data

In [ ]:
(model.predict(np.array([[-2, -1]])) > 0.5).astype(int).flatten()[0]

On constate qu'un réseau de neurones avec un seul neuronne ne peut apprendre qu'une fonction lineaire W * X +b
Donc résoudre notre problème de classification il faut que notre reseau de neurones apprenne une fonction non lineaire et pour faire cela on va combiner le resultat de plusieurs neuronnes ensemble.


<img src="">

In [ ]:
# Create the Sequential model
model =  tf.keras.Sequential()

# TODO

In [ ]:
# TODO compile the model

In [ ]:
# TODO fit the model

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plot_descision_boundary(model, X_train, y_train)

**On commence à apprendre des fonctions non linéaire mais on est encore loin du résultat attendu. On a joute donc encore plus de neurones.**

In [ ]:
# TODO create a deeper deep learning model with more neurons.
model =  tf.keras.Sequential()



In [ ]:
model.summary()

In [ ]:
# TODO compile the model

In [ ]:
# TOD fit the model

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
y_pred = (model.predict(X_test) > 0.5).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('confusion matrix', confusion_matrix(y_test, y_pred))
print('tn,fp,fn,tp \n', tn, fp, fn, tp)
print('precision', precision_score(y_test, y_pred))
print('recall', recall_score(y_test, y_pred))
print('f1 score', f1_score(y_test,y_pred))

In [ ]:
plot_descision_boundary(model, X_train, y_train)